# Titanic [kaggle](https://www.kaggle.com/c/titanic) task demo

## 1. Importing necessary libraries

In [6]:
import numpy as np
import pandas as pd
import grpc
import hydro_serving_grpc as hs
from grpc import ssl_channel_credentials

## 2. Reading data

In [26]:
train_df = pd.read_csv('../data/train.csv', header=0)

In [27]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [30]:
example = train_df.iloc[0]

## 3. Connecting to the server

In [9]:
channel = grpc.secure_channel("dev.k8s.hydrosphere.io", credentials=ssl_channel_credentials())
stub = hs.PredictionServiceStub(channel)

In [10]:
model = hs.ModelSpec(name="titanic", signature_name="infer")

## 4. Form tensors

In [36]:
pclass_tensor = hs.TensorProto(dtype=hs.DT_INT32, int_val=[example.Pclass])
sex_tensor = hs.TensorProto(dtype=hs.DT_STRING, string_val=[example.Sex.encode()])
age_tensor = hs.TensorProto(dtype=hs.DT_INT32, int_val=[int(example.Age)])
fare_tensor = hs.TensorProto(dtype=hs.DT_DOUBLE, double_val=[example.Fare])
parch_tensor = hs.TensorProto(dtype=hs.DT_INT32, int_val=[example.Parch])

## 5. Send request to model servable

In [37]:
request = hs.PredictRequest(model_spec=model, inputs={"pclass": pclass_tensor, 'sex': sex_tensor, 'age': age_tensor, 'fare': fare_tensor, 'parch': parch_tensor})
result = stub.Predict(request)
result_tensor = result.outputs.get('survived')

In [42]:
print(f'True answer: {example.Survived}')
print(f'Predicted answer: {result_tensor.int_val[0]}')

True answer: 0
Predicted answer: 0
